In [1]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import bambi as bmb
import xarray as xr
import random

In [2]:
df = pd.read_csv('../results/model/Nested_model/Ne_Pi_recomb.csv')
df = df[df['chr'] != 'chrX']


In [3]:
df['full_species']= df['genus']+'_'+df['species']

In [6]:
unique_Species = df['genus'].unique()


In [8]:
len(df['full_species'].unique())

187

In [10]:
unique_Species

array(['Allenopithecus', 'Allochrocebus', 'Alouatta', 'Aotus',
       'Arctocebus', 'Ateles', 'Cacajao', 'Callimico', 'Callithrix',
       'Cebuella', 'Cebus', 'Cercocebus', 'Cercopithecus', 'Cheirogaleus',
       'Cheracebus', 'Chiropotes', 'Chlorocebus', 'Colobus',
       'Daubentonia', 'Erythrocebus', 'Eulemur', 'Galago', 'Galagoides',
       'Gorilla', 'Hapalemur', 'Hoolock', 'Hylobates', 'Lagothrix',
       'Lemur', 'Leontocebus', 'Leontopithecus', 'Lepilemur',
       'Lophocebus', 'Loris', 'Macaca', 'Mandrillus', 'Mico',
       'Microcebus', 'Miopithecus', 'Mirza', 'Nomascus', 'Nycticebus',
       'Otolemur', 'Pan', 'Perodicticus', 'Pithecia', 'Plecturocebus',
       'Pongo', 'Prolemur', 'Rhinopithecus', 'Saguinus', 'Saimiri',
       'Sapajus', 'Semnopithecus', 'Trachypithecus', 'Varecia'],
      dtype=object)

In [33]:
df1 = pd.read_csv('../data/PDGP_metadata.txt')
df1 = df1.dropna()
df1['ref_assembly'].unique()


array(['Pithecia_pithecia', 'Callithrix_jacchus', 'Atele_fusciceps',
       'Cebus_albifrons', 'Aotus_nancymaae', 'Saguinus_midas',
       'Chlorocebus_aethiops', 'Cercopithecus_mitis',
       'Erythrocebus_patas', 'Nomascus_leucogenys', 'Mandrillus_sphinx',
       'Macaca_mulatta', 'Colobus_guereza', 'Rhinopithecus_roxellana',
       'Papio_anubis', 'Cercocebus_atys', 'Nycticebus_pygmaeus',
       'Propithecus_coquereli', 'Daubentonia_madagascariensis',
       'Lemur_catta-Thomas', 'Galago_moholi', 'Microcebus_murinus',
       'Gorilla_gorilla_gorilla', 'Pan_troglodytes', 'Pongo_abelii',
       'Sapajus_apella', 'Carlito_syrichta', 'Loris_tardigradus',
       'Otolemur_garnettii', 'Pongo_pygmaeus'], dtype=object)

In [43]:
df1['full_speices'] =  df1['Genus'] + '_' + df1['Species']
df1

,PDGP_ID,Genus,Species,FROH,Sex,ref_assembly,full_speices
0,PD_0001,Cheracebus,lugens,0.02829385,F,Pithecia_pithecia,Cheracebus_lugens
1,PD_0002,Callibella,humilis,0.00991667,M,Callithrix_jacchus,Callibella_humilis
2,PD_0003,Cacajao,hosomi,0.06922806,M,Pithecia_pithecia,Cacajao_hosomi
3,PD_0004,Ateles,belzebuth,0.08438225,F,Atele_fusciceps,Ateles_belzebuth
4,PD_0005,Alouatta,macconnelli,0.06254126,M,Atele_fusciceps,Alouatta_macconnelli
...,...,...,...,...,...,...,...
805,SAMN01920547,Pongo,pygmaeus,0.07808712,F,Pongo_pygmaeus,Pongo_pygmaeus
806,SAMN01920548,Pongo,pygmaeus,0.01311919,F,Pongo_pygmaeus,Pongo_pygmaeus
807,SAMN01920549,Pongo,pygmaeus,0.07752885,F,Pongo_pygmaeus,Pongo_pygmaeus
808,SAMN01920550,Pongo,pygmaeus,0.05289547,F,Pongo_pygmaeus,Pongo_pygmaeus


In [35]:
unique_Species2 = df1['full_speices'].unique()

In [36]:
len(unique_Species2)

216

In [51]:
union = []
intersect = []
for i in unique_Species:
    if i not in unique_Species2:
        intersect.append(i)
    else:
        union.append(i)
            

In [52]:
intersect

[]

In [48]:
df3 = df1[df1['full_speices'].isin(intersect)]

In [50]:
pd.set_option('display.max_rows', None)
print(df3)

         PDGP_ID           Genus          Species        FROH Sex  \
1        PD_0002      Callibella          humilis  0.00991667   M   
2        PD_0003         Cacajao           hosomi  0.06922806   M   
4        PD_0005        Alouatta      macconnelli  0.06254126   M   
5        PD_0006      Chiropotes        israelita  0.03880299   F   
8        PD_0009        Saguinus          inustus   0.3056544   M   
9        PD_0010        Cebuella     niveiventris  0.13701765   F   
11       PD_0012         Saimiri            ustus  0.04070355   F   
13       PD_0014   Plecturocebus          cupreus  0.02564819   F   
14       PD_0015        Pithecia       vanzolinii  0.04144865   M   
15       PD_0016       Lagothrix      lagothricha  0.02549358   M   
16       PD_0017     Leontocebus      nigricollis  0.04136473   F   
17       PD_0018         Cacajao   melanocephalus  0.03490799   M   
18       PD_0019         Cacajao           ayresi  0.20011467   F   
19       PD_0020         Cacajao  